In [1]:
from src.data import DataModule

dm = DataModule(
    r"C:\Users\user\data\dl_radiomics\preprocessed_3d",
    r"C:\Users\user\data\tables\lesion_followup_curated_v4.csv",
    "lung",
    "mst",
)
dm.setup()

Loading dataset: 100%|██████████| 56/56 [00:05<00:00, 10.18it/s]


In [2]:
dl = dm.train_dataloader()

In [3]:
x = next(iter(dl))

In [5]:
x ['patient']

['PREM_AM_115',
 'PREM_AM_115',
 'PREM_AM_118',
 'PREM_ZU_001',
 'PREM_ZU_004',
 'PREM_ZU_004',
 'PREM_ZU_008',
 'PREM_ZU_008',
 'PREM_ZU_012',
 'PREM_ZU_012',
 'PREM_ZU_017',
 'PREM_ZU_018',
 'PREM_ZU_018',
 'PREM_ZU_019',
 'PREM_ZU_019',
 'PREM_ZU_020']

In [80]:
for x in iter(dl):
    print(x['patient'])

['PREM_MS_011', 'PREM_MS_011', 'PREM_MS_011', 'PREM_MS_011', 'PREM_MS_011', 'PREM_MS_031', 'PREM_MS_031', 'PREM_MS_031', 'PREM_MS_031', 'PREM_MS_038', 'PREM_MS_038', 'PREM_MS_038', 'PREM_MS_038']
['PREM_MS_060', 'PREM_MS_060', 'PREM_MS_060', 'PREM_MS_060', 'PREM_MS_060', 'PREM_MS_082', 'PREM_MS_082', 'PREM_MS_082', 'PREM_MS_082', 'PREM_MS_082', 'PREM_MS_083', 'PREM_MS_103', 'PREM_MS_103', 'PREM_MS_103', 'PREM_MS_103', 'PREM_MS_103']
['PREM_MS_120', 'PREM_MS_125', 'PREM_MS_160', 'PREM_MS_160', 'PREM_MS_160', 'PREM_MS_160', 'PREM_MS_160', 'PREM_MS_172', 'PREM_MS_172', 'PREM_MS_172', 'PREM_MS_172', 'PREM_MS_195', 'PREM_MS_195', 'PREM_MS_195', 'PREM_MS_195']
['PREM_MS_234', 'PREM_MS_234', 'PREM_MS_236', 'PREM_MS_236', 'PREM_MS_236', 'PREM_MS_236', 'PREM_MS_236', 'PREM_MS_237', 'PREM_MS_237', 'PREM_MS_237', 'PREM_MS_237', 'PREM_MS_237']


In [73]:
x['patient']

['PREM_MS_011',
 'PREM_MS_011',
 'PREM_MS_011',
 'PREM_MS_011',
 'PREM_MS_011',
 'PREM_MS_031',
 'PREM_MS_031',
 'PREM_MS_031',
 'PREM_MS_031',
 'PREM_MS_038',
 'PREM_MS_038',
 'PREM_MS_038',
 'PREM_MS_038']